In [1]:
import pandas as pd

competitors_data = pd.read_csv('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/csv_files/competitors_details.csv')
competitors_data

,competitor_id,competitor_name,competitor_country,competitor_country_code,competitor_abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
3,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
4,sr:competitor:36593,"Krawietz, Kevin",Germany,DEU,KRA
...,...,...,...,...,...
995,sr:competitor:831284,"Kuramochi, Miho",Japan,JPN,KUR
996,sr:competitor:203959,"Desai, Zeel",India,IND,DES
997,sr:competitor:784280,"Prozorova, Tatiana",Neutral,NaN,PRO
998,sr:competitor:840734,"Walker, Brandy",USA,USA,WAL


save data in postgresql database

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


Creating DataBase:

In [3]:
pip install toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


Creating competitors Table

In [5]:
competitors_create_table_sql = """create table if not exists competitors_table(
competitor_id varchar(50) PRIMARY KEY, 
competitor_name varchar(100) NOT NULL,
competitor_country varchar(100) NOT NULL,
competitor_country_code char(3) NOT NULL,
competitor_abbreviation varchar(10) NOT NULL)"""

#execute:
mycursor.execute(competitors_create_table_sql)

Fetching Column Name from competitors_table:

In [6]:
column_name = """SELECT column_name FROM 
        information_schema.columns WHERE table_name = 'competitors_table';"""
mycursor.execute(column_name)
fetching_column_names = mycursor.fetchall()
fetching_column_names 

[('competitor_id',),
 ('competitor_name',),
 ('competitor_country',),
 ('competitor_country_code',),
 ('competitor_abbreviation',)]

insert values into categories table:

In [7]:
competitors_insert_table_sql = """Insert into competitors_table(
competitor_id, 
competitor_name,
competitor_country,
competitor_country_code,
competitor_abbreviation
) 
values (%s,%s,%s,%s,%s) ON CONFLICT (competitor_id) DO NOTHING"""


#ON CONFLICT (cat_id) DO NOTHING
#“If there’s a conflict — like trying to insert a row with a duplicate primary key or unique value — then just skip that row and move on. 
# Don’t insert it, and don’t throw an error.”

In [8]:
for index, row in competitors_data.iterrows():
    print(row) #We will get the o/p in the format of dictionary with column names as keys.

    print("Inserting:", tuple(row))  #The iter row (only values) will converts into python tuple 
    print("----------------------\n")
    mycursor.execute(competitors_insert_table_sql, tuple(row))
print("Data inserted  into the categories_table using iterrows()")


competitor_id              sr:competitor:49363
competitor_name                    Pavic, Mate
competitor_country                     Croatia
competitor_country_code                    HRV
competitor_abbreviation                    PAV
Name: 0, dtype: object
Inserting: ('sr:competitor:49363', 'Pavic, Mate', 'Croatia', 'HRV', 'PAV')
----------------------

competitor_id                    sr:competitor:51836
competitor_name            Arevalo-Gonzalez, Marcelo
competitor_country                       El Salvador
competitor_country_code                          SLV
competitor_abbreviation                          ARE
Name: 1, dtype: object
Inserting: ('sr:competitor:51836', 'Arevalo-Gonzalez, Marcelo', 'El Salvador', 'SLV', 'ARE')
----------------------

competitor_id              sr:competitor:14898
competitor_name              Heliovaara, Harri
competitor_country                     Finland
competitor_country_code                    FIN
competitor_abbreviation                    HEL
Nam

Fetching Data from competitors_table:

In [9]:
#execute:
mycursor.execute("Select * from competitors_table")

fetching_Data = mycursor.fetchall()
fetching_Data

[('sr:competitor:49363', 'Pavic, Mate', 'Croatia', 'HRV', 'PAV'),
 ('sr:competitor:51836',
  'Arevalo-Gonzalez, Marcelo',
  'El Salvador',
  'SLV',
  'ARE'),
 ('sr:competitor:14898', 'Heliovaara, Harri', 'Finland', 'FIN', 'HEL'),
 ('sr:competitor:637970', 'Patten, Henry', 'Great Britain', 'GBR', 'PAT'),
 ('sr:competitor:36593', 'Krawietz, Kevin', 'Germany', 'DEU', 'KRA'),
 ('sr:competitor:52293', 'Putz, Tim', 'Germany', 'DEU', 'PUT'),
 ('sr:competitor:15568', 'Granollers, Marcel', 'Spain', 'ESP', 'GRA'),
 ('sr:competitor:16160', 'Zeballos, Horacio', 'Argentina', 'ARG', 'ZEB'),
 ('sr:competitor:95801', 'Vavassori, Andrea', 'Italy', 'ITA', 'VAV'),
 ('sr:competitor:15310', 'Bolelli, Simone', 'Italy', 'ITA', 'BOL'),
 ('sr:competitor:87690', 'Thompson, Jordan', 'Australia', 'AUS', 'THO'),
 ('sr:competitor:124658', 'Purcell, Max', 'Australia', 'AUS', 'PUR'),
 ('sr:competitor:16608', 'Mektic, Nikola', 'Croatia', 'HRV', 'MEK'),
 ('sr:competitor:59131', 'Glasspool, Lloyd', 'Great Britain', 'GBR

In [10]:
mycursor.close()
my_db_connection.close()